In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, BatchNormalization, Activation, MaxPool2D, GlobalAveragePooling2D, Add, Dropout
from tensorflow.keras import Model
## resnet paper - https://arxiv.org/pdf/1512.03385.pdf
## issue may be images are too small to run convolutions,
## but residual blocks should provide some accuracy bumps

class ResidualBlock(Model):
    def __init__(self, channel_in = 64, channel_out = 256):
        super().__init__()
        
        channel = channel_out // 4
        self.dropout = Dropout(.2)
        self.conv1 = Conv2D(channel, kernel_size = (1, 1), padding = "same")
        self.bn1 = BatchNormalization()
        self.av1 = Activation(tf.nn.relu)
        self.conv2 = Conv2D(channel, kernel_size = (3, 3), padding = "same")
        self.bn2 = BatchNormalization()
        self.av2 = Activation(tf.nn.relu)
        self.conv3 = Conv2D(channel_out, kernel_size = (1, 1), padding = "same")
        self.bn3 = BatchNormalization()
        self.shortcut = self._shortcut(channel_in, channel_out)
        self.add = Add()
        self.av3 = Activation(tf.nn.relu)
        
    def call(self, x):
        h = self.conv1(x)
        h = self.bn1(h)
        h = self.av1(h)
        h = self.conv2(h)
        h = self.bn2(h)
        h = self.av2(h)
        h = self.conv3(h)
        h = self.bn3(h)
        shortcut = self.shortcut(x)
        h = self.add([h, shortcut])
        y = self.av3(h)
        return y
    
    def _shortcut(self, channel_in, channel_out):
        if channel_in == channel_out:
            return lambda x : x
        else:
            return self._projection(channel_out)
        
    def _projection(self, channel_out):
        return Conv2D(channel_out, kernel_size = (1, 1), padding = "same")
           
class LightNet18(Model):
    def __init__(self, input_shape, output_dim):
        super().__init__()                
        self._layers = [
            # conv1
            Conv2D(64, input_shape = input_shape, kernel_size = (7, 7), strides=(2, 2), padding = "same"),
            BatchNormalization(),
            Activation(tf.nn.relu),
            # conv2_x
            MaxPool2D(pool_size = (3, 3), strides = (2, 2), padding = "same"),
            ResidualBlock(64, 64),
            # conv3_x
            Conv2D(128, kernel_size = (1, 1), strides=(2, 2)),
            ResidualBlock(128, 128),
            # conv4_x
            Conv2D(256, kernel_size = (1, 1), strides=(2, 2)),
            ResidualBlock(256, 256),
            # conv5_x
            Conv2D(512, kernel_size = (1, 1), strides=(2, 2)),
            ResidualBlock(512, 512),
            # last part
            GlobalAveragePooling2D(),
            Dense(1000, activation = tf.nn.relu),
            Dense(output_dim, activation = tf.nn.softmax)
        ]
        
    def call(self, x):
        for layer in self._layers:
            if isinstance(layer, list):
                for l in layer:
                    x = l(x)    
            else:
                x = layer(x)
        return x
       
    
model = LightNet18((28, 28, 1), 10)
model.build(input_shape = (None, 28, 28, 1))

2022-04-25 10:56:58.627486: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-25 10:56:58.644241: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2397400000 Hz
2022-04-25 10:56:58.649011: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x557dfd6ee630 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-04-25 10:56:58.649043: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [2]:
model.summary()

Model: "light_net18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  3200      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
residual_block (ResidualBloc multiple                  4832      
_________________________________________________________________
conv2d_4 (Conv2D)            multiple                  8320      
_________________________________________________________________
residual_block_1 (ResidualBl multiple                  

In [3]:
import os
checkpoint_filepath = "lightnet18/cp.ckpt"
model.load_weights(checkpoint_filepath).expect_partial()

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
x_train = tf.expand_dims(x_train, axis=3)
x_test = tf.expand_dims(x_test, axis=3)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
print(model.evaluate(x_test, y_test))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

313/313 [==============================] - 12s 38ms/step - loss: 0.2864 - accuracy: 0.9087
[0.28641825914382935, 0.9086999893188477]
